# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(w_i|x)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1. Bayes Rule 증명하는 식으로 작성  
: Naive Bayes classifier는 입력 𝑥에 대해 각 클래스 𝑤𝑖의 사후 확률을 계산하여 가장 높은 확률을 가진 클래스를 선택한다. 이때 사후 확률은 각 클래스에 대한 x의 조건부확률인 우도를 𝑥의 확률로 나누어 구한다. x의 확률은 베이지안 정리에 따라 각 클래스에 대한 우도와 사전확률을 곱한 값을 더해 구할 수 있다.

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [2]:
# pip install pandas
import pandas as pd
import re

In [3]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [4]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [5]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [7]:
df

,review,sentiment
0,love great exceeded expectations,positive
1,worst purchase have ever made completely useless,negative
2,average nothing special but not terrible either,neutral
3,great who can help but love design highly reco...,positive
4,terrible experience will never buy from poor b...,negative
5,acceptable but expected better not just accept...,neutral
6,absolutely wonderful very satisfied with great,positive
7,quality poor broke after one use terrible enough,negative
8,acceptable for price but there better options ...,neutral
9,great quality fast shipping with wonderful love,positive


In [10]:
# 사전 확률 구하는 코드를 작성해주세요.

# 사전확률 -> countvectoriser library
prior = df['sentiment'].value_counts(normalize=True)
prior

,proportion
sentiment,
positive,0.4
negative,0.3
neutral,0.3


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [12]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [13]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [14]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [28]:
frequency_matrix.columns

Index(['sentiment', 'absolutely', 'acceptable', 'after', 'again', 'average',
       'better', 'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help', 'highly',
       'just', 'love', 'made', 'never', 'not', 'nothing', 'one', 'options',
       'out', 'poor', 'price', 'purchase', 'quality', 'recommend', 'satisfied',
       'shipping', 'special', 'terrible', 'there', 'use', 'useless', 'very',
       'who', 'will', 'with', 'wonderful', 'worst'],
      dtype='object')

In [54]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요

# 각 클래스별로 단어 빈도 합계 계산
class_word_counts = frequency_matrix.groupby('sentiment').sum()
class_word_counts

,absolutely,acceptable,after,again,average,better,brand,broke,but,buy,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
sentiment,,,,,,,,,,,,,,,,,,,,,
negative,0,0,1,1,0,0,1,1,0,1,...,2,0,1,1,0,0,1,0,0,1
neutral,0,3,0,0,1,2,0,0,3,0,...,1,2,0,0,0,0,0,0,0,0
positive,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,2,2,0


In [55]:
# 각 단어 클래스별 조건부 확률
conditional_probabilities = class_word_counts.div(class_word_counts.sum(axis=1), axis=0)
conditional_probabilities

,absolutely,acceptable,after,again,average,better,brand,broke,but,buy,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
sentiment,,,,,,,,,,,,,,,,,,,,,
negative,0.000000,0.000,0.041667,0.041667,0.000000,0.000000,0.041667,0.041667,0.000000,0.041667,...,0.083333,0.000000,0.041667,0.041667,0.000000,0.000000,0.041667,0.000000,0.000000,0.041667
neutral,0.000000,0.125,0.000000,0.000000,0.041667,0.083333,0.000000,0.000000,0.125000,0.000000,...,0.041667,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
positive,0.038462,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.038462,0.038462,0.000000,0.076923,0.076923,0.000000


독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [61]:
# 가능도 구하는 코드를 작성해주세요.

def calculate_likelihood(words, conditional_probabilities):
    likelihoods = {
        'positive': [],
        'negative': [],
        'neutral': []
    }

    for word in words:
        for sentiment in likelihoods.keys():
            if word in conditional_probabilities.columns:
                likelihood = conditional_probabilities.loc[sentiment, word]
            else:
                likelihood = 0  # 없는 단어일 때...
            likelihoods[sentiment].append(likelihood)

    return likelihoods

# P(love,great,awesome| . )
first = ['love', 'great', 'awesome']
likelihoods_first = calculate_likelihood(first, conditional_probabilities)

print("target_review1")
print("Likelihoods for 'positive':", likelihoods_first['positive'])
print("Likelihoods for 'negative':", likelihoods_first['negative'])
print("Likelihoods for 'neutral':", likelihoods_first['neutral'])


print("\n")

# P(terrible,not,never| . )
second = ['terrible', 'not', 'never']
likelihoods_second = calculate_likelihood(second, conditional_probabilities)

print("target_review2")
print("Likelihoods for 'positive':", likelihoods_second['positive'])
print("Likelihoods for 'negative':", likelihoods_second['negative'])
print("Likelihoods for 'neutral':", likelihoods_second['neutral'])

target_review1
Likelihoods for 'positive': [0.11538461538461539, 0.15384615384615385, 0]
Likelihoods for 'negative': [0.0, 0.0, 0]
Likelihoods for 'neutral': [0.0, 0.0, 0]


target_review2
Likelihoods for 'positive': [0.0, 0.0, 0.0]
Likelihoods for 'negative': [0.08333333333333333, 0.0, 0.041666666666666664]
Likelihoods for 'neutral': [0.041666666666666664, 0.08333333333333333, 0.0]


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [62]:
import numpy as np
# 최종 확률 구하는 코드를 작성해주세요.

def calculate_posterior(likelihoods, prior):
    posterior = {}
    for sentiment in likelihoods.keys():
        likelihood = np.prod(likelihoods[sentiment])
        posterior[sentiment] = likelihood * prior[sentiment]

    total = sum(posterior.values())
    if total > 0:
        for sentiment in posterior.keys():
            posterior[sentiment] /= total
    return posterior

# 첫번째
posterior_first = calculate_posterior(likelihoods_first, prior)

# P(positive|target_review1)
posterior_first_positive = posterior_first['positive']
# P(negative|target_review1)
posterior_first_negative = posterior_first['negative']
# P(neutral|target_review1)
posterior_first_neutral = posterior_first['neutral']

print("P(positive|target_review1):", posterior_first_positive)
print("P(negative|target_review1):", posterior_first_negative)
print("P(neutral|target_review1):", posterior_first_neutral)

print("\n")

# 두번째
posterior_second = calculate_posterior(likelihoods_second, prior)

# P(positive|target_review2)
posterior_second_positive = posterior_second['positive']
# P(negative|target_review2)
posterior_second_negative = posterior_second['negative']
# P(neutral|target_review2)
posterior_second_neutral = posterior_second['neutral']

print("P(positive|target_review2):", posterior_second_positive)
print("P(negative|target_review2):", posterior_second_negative)
print("P(neutral|target_review2):", posterior_second_neutral)

P(positive|target_review1): 0.0
P(negative|target_review1): 0.0
P(neutral|target_review1): 0.0


P(positive|target_review2): 0.0
P(negative|target_review2): 0.0
P(neutral|target_review2): 0.0


A2-1.   
Target review1의 분류 결과: target_review1에 포함된 단어 모두 특정 클래스에 할당되지 않음   
Target review2의 분류 결과: target_review2에 포함된 단어 모두 특정 클래스에 할당되지 않음

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. 어떤 단어가 등장하지 않으면 그 클래스의 가능도가 0이 되어버리는 희소성 문제가 있었다. 이것이 사후 확률 계산에 큰 영향을 미쳤다. 라플라스 스무딩을 통해 모든 단어가 최소한 한 번 이상 나타난 것으로 취급하는 등의 처치가 필요해 보인다.